Exercise: Secrets
----------------------

The use of Secrets will be demonstrated using MySQL as an example.

First, the secrets must be created using `kubectl`:

In [ ]:
! kubectl create secret generic mysql-pass --from-literal=password=admin
! kubectl create secret generic mysql-user-pass --from-literal=password=secret

In [ ]:
! kubectl get secrets

The environment variables in the YAML file must then be set in such a way that they access the secret values

        env:
        - name: MYSQL_ROOT_PASSWORD
          valueFrom:
           secretKeyRef:
            name: mysql-pass
            key: password
            
        - name: MYSQL_PASSWORD
          valueFrom:
           secretKeyRef:
            name: mysql-user-pass
            key: password

The other values, such as username, can be set in the YAML file:

        env:
        - name: MYSQL_USER
          value: test
        - name: MYSQL_DATABASE
          value: test 
          
**Note**: The information about which environment variables we can set is in the documentation of the container image [mysql](https://hub.docker.com/_/mysql)

In [ ]:
! kubectl apply -f 09-7-Secrets/mysql.yaml

For testing we use [adminer](https://www.adminer.org/de/).

The server (hostname) is `mysql`, username, password and database result from the previous configuration.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml
! echo "http://$(cat ~/work/server-ip):$(kubectl get service adminer -o=jsonpath='{ .spec.ports[0].nodePort }')/adminer"

- - -

### Security

Since [Secrets](https://kubernetes.io/docs/concepts/configuration/secret/) is a separate resource, we can secure it separately. For example, the operating company is not allowed to see them, despite access to the dashbard.

However, if the application/database needs these as plain text environment variables, security is no longer given, at the latest when changing to the container.


In [ ]:
! kubectl exec deployment/mysql -- printenv | grep MYSQL

#### alternatives

Then alternatives are required, e.g.

* [Hashicorp Vault](https://www.vaultproject.io/)
* [Bitnami Sealed Secrets](https://github.com/bitnami-labs/sealed-secrets)

Quelle: [5 GitOps Best Practices](https://blog.argoproj.io/5-gitops-best-practices-d95cb0cbe9ff)

- - -

Clean up

In [ ]:
! kubectl delete -f 09-7-Secrets/mysql.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml
! kubectl delete secret mysql-pass
! kubectl delete secret mysql-user-pass